# SetFit for Multilabel Text Classification

In this notebook, we'll learn how to do few-shot text classification on a multilabel dataset with SetFit.

## Setup

If you're running this Notebook on Colab or some other cloud platform, you will need to install the `setfit` library. Uncomment the following cell and run it:

In [1]:
# %pip install setfit

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS, which you can do by uncommenting and running following command:

In [3]:
# !apt install git-lfs

Finally, you may need to configue Git on your system by providing details about who you are:

In [ ]:
# !git config --global user.email "you@example.com"
# !git config --global user.name "Your Name"

This notebook is designed to work with any multiclass [text classification dataset](https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads) and pretrained [Sentence Transformer](https://huggingface.co/models?library=sentence-transformers&sort=downloads) on the Hub. Change the values below to try a different dataset / model!

To be able to share your model with the community, there are a few more steps to follow.

First, you have to store your authentication token from the Hugging Face Hub (sign up [here](https://huggingface.co/join) if you haven't already!). To do so, execute the following cell and input an [access token](https://huggingface.co/docs/hub/security-tokens) associated with your account:

In [37]:
from scraper.scrape_utils import wayback_scraper, get_article
import numpy as np
import pandas as pd

In [29]:
df = pd.read_csv('train_data/v1/data.csv')

In [30]:
di = {True: 1, False: 0}

In [31]:
df = df.replace(di)

In [32]:
df

,url,t_mil,t_loc,t_milcas,t_civcas,t_isis_vic,date
0,https://www.cnn.com/2019/03/23/middleeast/isis...,1,1,0,0,0,03/23/19
1,https://www.cnn.com/2017/06/25/asia/philippine...,1,1,1,0,1,08/24/17
2,https://www.cnn.com/2017/06/06/middleeast/raqq...,1,1,0,0,1,06/06/17
3,https://fox2now.com/news/isis-in-the-crosshair...,1,1,0,0,0,08/16/16
4,https://www.cnn.com/2016/12/12/middleeast/palm...,1,1,1,0,1,09/12/16
5,https://www.fox13seattle.com/news/obama-were-i...,1,1,1,1,0,11/09/14
6,https://www.nbcnews.com/storyline/isis-terror/...,1,1,0,0,0,12/21/15
7,http://fingfx.thomsonreuters.com/gfx/rngs/PHIL...,1,1,1,0,1,05/25/17
8,https://www.defense.gov/News/News-Stories/Arti...,1,1,0,0,1,03/13/17
9,https://www.cnbc.com/2014/10/25/iraq-notches-v...,1,1,1,0,0,10/25/14


In [38]:
text = []
for url in df['url'].array:
    text.append(get_article(url))

In [48]:
text[7]

''

In [43]:
df['text'] = text

In [52]:
df['text'].replace('', np.nan, inplace=True)

In [55]:
df.dropna(subset=['text'], inplace=True)

In [62]:
df

,url,t_mil,t_loc,t_milcas,t_civcas,t_isis_vic,date,text
0,https://www.cnn.com/2019/03/23/middleeast/isis...,1,1,0,0,0,03/23/19,Eastern Syria CNN —\n\nISIS has lost its final...
1,https://www.cnn.com/2017/06/25/asia/philippine...,1,1,1,0,1,08/24/17,"Iligan, Philippines CNN —\n\nDuring the rainy ..."
2,https://www.cnn.com/2017/06/06/middleeast/raqq...,1,1,0,0,1,06/06/17,Story highlights US-backed forces have been pu...
3,https://fox2now.com/news/isis-in-the-crosshair...,1,1,0,0,0,08/16/16,This is an archived article and the informatio...
4,https://www.cnn.com/2016/12/12/middleeast/palm...,1,1,1,0,1,09/12/16,Story highlights ISIS in control of ancient ci...
5,https://www.fox13seattle.com/news/obama-were-i...,1,1,1,1,0,11/09/14,article\n\n\n\n\n\n(CNN) -- The decision to in...
6,https://www.nbcnews.com/storyline/isis-terror/...,1,1,0,0,0,12/21/15,A group of five men crouch behind a berm. Peri...
8,https://www.defense.gov/News/News-Stories/Arti...,1,1,0,0,1,03/13/17,Iraqi security forces are battling Islamic Sta...
9,https://www.cnbc.com/2014/10/25/iraq-notches-v...,1,1,1,0,0,10/25/14,Iraqi government forces and Shi'ite militias s...
10,https://www.cnbc.com/2015/04/07/isis-in-damasc...,1,1,0,0,1,04/07/15,The arrival of ISIS fighters in Syria's capita...


In [65]:
from datasets import load_dataset, Dataset

model_id = "sentence-transformers/paraphrase-mpnet-base-v2"
#dataset = load_dataset("ethos", "multilabel")
dataset = Dataset.from_pandas(df)

In [66]:
dataset

Dataset({
    features: ['url', 't_mil', 't_loc', 't_milcas', 't_civcas', 't_isis_vic', 'date', 'text', '__index_level_0__'],
    num_rows: 19
})

## Loading and sampling the dataset

Most datasets on the Hub have many more labeled examples than those one encounters in few-shot settings. To simulate the effect of training on a limited number of examples, let's subsample the training set to have at least 8 labeled examples per feature.

Note that if your dataset has differently formatted labels, you may need to adapt this section.

In [70]:
import numpy as np

features = dataset.column_names
features.remove("text")
features.remove("url")
features.remove("date")
features.remove("__index_level_0__")
features

['t_mil', 't_loc', 't_milcas', 't_civcas', 't_isis_vic']

In [72]:
dataset[4]

{'url': 'https://www.cnn.com/2016/12/12/middleeast/palmyra-syria-isis-russia/index.html',
 't_mil': 1,
 't_loc': 1,
 't_milcas': 1,
 't_civcas': 0,
 't_isis_vic': 1,
 'date': '09/12/16',
 'text': 'Story highlights ISIS in control of ancient city, state media reports Militant group in 2015 blew up ancient treasures there\n\nCNN —\n\nISIS forces have retaken the ancient city of Palmyra in Syria, according to Syrian government media, the ISIS media wing and a human rights monitor.\n\nSyrian news agency SANA reported that over 4,000 militants swarmed the city from “various directions,” despite having suffered heavy losses from bombardments by the Syrian air force. The Russian Defense Ministry had earlier reported that its aircraft had also taken part in the air campaign.\n\nThe UK-based Syrian Observatory for Human Rights (SOHR) confirmed that Palmyra had fallen to ISIS on Sunday after Syrian armed forces pulled out from the desert city, the organization said.\n\n“Despite the ongoing air r

In [8]:
#num_samples = 8
#samples = np.concatenate(
#    [np.random.choice(np.where(dataset["train"][f])[0], num_samples) for f in features]
#)

We encode the emotions in a single `'label'` feature. 

In [73]:
def encode_labels(record):
    return {"labels": [record[feature] for feature in features]}


dataset = dataset.map(encode_labels)

Map: 100%|███████| 19/19 [00:00<00:00, 1063.61 examples/s]


Next, we use the samples we selected as our training set, and the others as our test set (since the ethos dataset does not have a test split on the hub).

Here we have 64 total examples to train with since the `ethos` dataset has 8 classes.

In [93]:
train_dataset = dataset.select(range(14))
eval_dataset = dataset.select(range(14,19))

Okay, now we have the dataset, let's load and train a model!

## Fine-tuning the model

To train a SetFit model, the first thing to do is download a pretrained checkpoint from the Hub. We can do so by using the `from_pretrained()` method associated with the `SetFitModel` class.

**Note that the `multi_target_strategy` parameter here signals to both the model and the trainer to expect a multi-labelled dataset.**

In [96]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained(model_id, multi_target_strategy="one-vs-rest")

2023-10-17 18:44:08.365759: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 18:44:09.661839: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Here, we've downloaded a pretrained Sentence Transformer from the Hub and added a logistic classification head to the create the SetFit model. As indicated in the message, we need to train this model on some labeled examples. We can do so by using the `SetFitTrainer` class as follows:

In [97]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    column_mapping={"text": "text", "labels": "label"},
)

The main arguments to notice in the trainer is the following:

* `loss_class`: The loss function to use for contrastive learning with the Sentence Transformer body
* `num_iterations`: The number of text pairs to generate for contrastive learning
* `column_mapping`: The `SetFitTrainer` expects the inputs to be found in a `text` and `label` column. This mapping automatically formats the training and evaluation datasets for us.

Now that we've created a trainer, we can train it!

In [99]:
trainer.train()

Applying column mapping to training dataset
Generating Training Pairs: 100%|█| 20/20 [00:00<00:00, 220
***** Running training *****
  Num examples = 1100
  Num epochs = 1
  Total optimization steps = 69
  Total train batch size = 16
Epoch:   0%|                        | 0/1 [00:00<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /home/paul/.local/lib/python3.10/site-packages/setfit/trainer.py:395 in train                    │
│                                                                                                  │
│   392 │   │   │   logger.info(f"  Total train batch size = {batch_size}")                        │
│   393 │   │   │                                                                                  │
│   394 │   │   │   warmup_steps = math.ceil(total_train_steps * self.warmup_proportion)           │
│ ❱ 395 │   │   │   self.model.model_body.fit(                                                     │
│   396 │   │   │   │   train_objectives=[(train_dataloader, train_loss)],                         │
│   397 │   │   │   │   epochs=num_epochs,                                                         │
│   398 │   │   │   │   optimizer_params={"lr": learning_rate},                                    │
│                                                                                                  │
│ /home/paul/.local/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:721  │
│ in fit                                                                                           │
│                                                                                                  │
│   718 │   │   │   │   │   │                                                                      │
│   719 │   │   │   │   │   │   skip_scheduler = scaler.get_scale() != scale_before_step           │
│   720 │   │   │   │   │   else:                                                                  │
│ ❱ 721 │   │   │   │   │   │   loss_value = loss_model(features, labels)                          │
│   722 │   │   │   │   │   │   loss_value.backward()                                              │
│   723 │   │   │   │   │   │   torch.nn.utils.clip_grad_norm_(loss_model.parameters(), max_grad   │
│   724 │   │   │   │   │   │   optimizer.step()                                                   │
│                                                                                                  │
│ /home/paul/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1194 in _call_impl     │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /home/paul/.local/lib/python3.10/site-packages/sentence_transformers/losses/CosineSimilarityLoss │
│ .py:39 in forward                                                                                │
│                                                            

The final step is to compute the model's performance using the `evaluate()` method. The default metric measures 'subset accuracy', which measures the fraction of samples where we predict all 8 labels correctly.

In [15]:
metrics = trainer.evaluate()
metrics

Applying column mapping to evaluation dataset
***** Running evaluation *****


{'accuracy': 0.30319148936170215}

And once the model is trained, you can push it to the Hub:

In [ ]:
trainer.push_to_hub(f"setfit-ethos-multilabel-example")

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `your-username/the-name-you-picked` so for instance:

In [12]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("lewtun/setfit-ethos-multilabel-example")

Downloading:   0%|          | 0.00/668 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/668 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/712k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.8k [00:00<?, ?B/s]

Run inference. As is usual in toxicity models, it tends to think any mention of topics such as race or gender are negative.

In [13]:
preds = model(
    [
        "Jewish people often don't eat pork.",
        "Is this lipstick suitable for people with dark skin?"
    ]
)
preds

array([[0, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0]])

In [14]:
# Show predicted labels, requires you to have stored the 'features' somewhere
[[f for f, p in zip(features, ps) if p] for ps in preds]

[['religion'], ['directed_vs_generalized']]